In [1]:
import pandas as pd

In [3]:
raw = pd.read_csv('./data/bank-additional-full.csv', sep = ';')
raw.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [5]:
# one-hot-encoding
data = pd.get_dummies( raw, columns = ['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome'] )

In [6]:
# make IDs
data['id'] = range(len(data))

In [20]:
# choose 30,000 randomly and set it as a train set
train = data.sample(30000, replace = False, random_state = 42).reset_index(drop = True)

In [21]:
# set the rest data as a test set
test = data.loc[ ~data['id'].isin(train['id']) ].reset_index(drop = True)

In [22]:
data.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'mon

In [23]:
input_var = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success']

# Xgboost

In [24]:
from xgboost import XGBClassifier

In [31]:
a = {'yes' : 1, 'no' : 0}
train['y'] = train['y'].map(a)
test['y'] = test['y'].map(a)

In [33]:
# train xgboost model with the train data set
xgb = XGBClassifier( n_estimators = 300, learning_rate = 0.1)
xgb.fit(train[input_var], train['y'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [34]:
# lets predict
predictions = xgb.predict( test[input_var] )

In [35]:
# check the accuracy
(pd.Series(predictions) == test['y']).mean()

0.9174115123346442

what is the best n_estimatior?

In [37]:
for n in [100, 200, 300, 400, 500, 600, 700, 800, 900]:
    xgb = XGBClassifier(n_estimators = n, learning_rate = 0.5)
    xgb.fit( train[input_var], train['y'] )
    predictions = xgb.predict(test[input_var])
    print( f"n = {n}, {(pd.Series(predictions) == test['y']).mean()}" )

n = 100, 0.9096353235609582
n = 200, 0.907847693957812
n = 300, 0.90757954951734
n = 400, 0.9083839828387558
n = 500, 0.9093671791204863
n = 600, 0.9080264569181266
n = 700, 0.9076689309974973
n = 800, 0.9072220235967108
n = 900, 0.90757954951734
